In [14]:
import trimesh

scene = trimesh.Scene()

In [ ]:
import my_code.diffusion_training_sign_corr.data_loading as data_loading

single_dataset, test_dataset = data_loading.get_val_dataset(
    'DT4D_inter_pair', 'test', 200, preload=False, return_evecs=True
    )

In [ ]:
with open('/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/'
          # 'single_64_2-2ev_64-128-128_remeshed/eval/epoch_99/SHREC19_r_pair-test/taubin-5/log_taubin-5.txt', 'r') as f:
        'single_64_1-2ev_64-128-128_remeshed_fixed/eval/epoch_99/DT4D_inter_pair-test/no_smoothing/log_no_smoothing.txt', 'r') as f:
    lines = f.readlines()
len(lines)

# try with smoothing

In [ ]:
pairzo_errs = []

search_str = 'Geo error est pairzo mean: '

example_str = 'Geo error est pairzo mean: 2.233'

for line in lines:
    if 'Geo error est pairzo mean: ' in line:
        pairzo_errs.append(float(line.split(search_str)[1].strip()))
        
len(pairzo_errs)

In [6]:
shape_ids_first = []
shape_ids_second = []

# for corr_file in test_dataset.corr_files:
    # 'data/SHREC19_r/corres/12_1.map'
    # id_1 = int(corr_file.split('/')[-1].split('_')[0]) - 1
    # id_2 = int(corr_file.split('/')[-1].split('_')[1].split('.')[0]) - 1
    
    # shape_ids_first.append(id_1)
    # shape_ids_second.append(id_2)
    
for combination in test_dataset.combinations:
    shape_ids_first.append(combination[0])
    shape_ids_second.append(combination[1])

In [8]:
import torch

# set to 0, for each unique shape id

unique_shapes = set(shape_ids_first)

errs_by_shape = torch.zeros(len(single_dataset))
times_by_shape = torch.zeros(len(single_dataset))

for i in range(len(shape_ids_first)):
    # print(i)
    errs_by_shape[shape_ids_first[i]] += pairzo_errs[i]
    errs_by_shape[shape_ids_second[i]] += pairzo_errs[i]
    
    times_by_shape[shape_ids_first[i]] += 1
    times_by_shape[shape_ids_second[i]] += 1
    
errs_by_shape /= times_by_shape    


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

axs[0].plot(errs_by_shape.numpy())
axs[1].hist(errs_by_shape.numpy(), bins=50)

# plt.label('Errors by shape, no smoothing')
# make a title above all subplots
fig.suptitle('Errors by shape, taubin-5')

plt.show()

In [ ]:
sorted_errs, sorted_ids = torch.sort(errs_by_shape, descending=True)

print(sorted_errs)
print(sorted_ids)

# get the 10 worst shapes, without nans 
worst_shapes = sorted_ids[~torch.isnan(sorted_errs)][:40]
print(worst_shapes)
print(errs_by_shape[worst_shapes])


In [ ]:
# add 4 random training shapes to trimesh scene

# np.random.shuffle(train_shapes)
scene.geometry.clear()

# idx_train = errs_by_shape.argsort(descending=True)[1:]

# for i, idx in enumerate(worst_shapes):
for i, idx in enumerate([61]):
    scene.add_geometry(trimesh.Trimesh(
        vertices=single_dataset[idx]['verts'] + torch.tensor([i, 0, 0]),
        faces=single_dataset[idx]['faces']))
    
axis = trimesh.creation.axis(axis_length=1)
scene.add_geometry(axis)
scene.show()

# Try to smooth the mesh

In [ ]:
mesh_65 = trimesh.Trimesh(vertices=single_dataset[65]['verts'], faces=single_dataset[65]['faces'])

scene.geometry.clear()
scene.add_geometry(mesh_65)
scene.show()

In [ ]:
import numpy as np

mesh_65_smoothed = trimesh.Trimesh(vertices=single_dataset[65]['verts'], faces=single_dataset[65]['faces'])

trimesh.smoothing.filter_laplacian(mesh_65_smoothed, lamb=0.5, iterations=20)
# trimesh.smoothing.filter_taubin(mesh_65_smoothed, lamb=0.5, iterations=20)
mesh_65_smoothed.vertices += np.array([1, 0, 0])

scene.geometry.clear()
axis = trimesh.creation.axis(axis_length=1)
scene.add_geometry(axis)

scene.add_geometry(mesh_65)
scene.add_geometry(mesh_65_smoothed)

scene.show()